In [24]:
import pyttsx3
import os
import subprocess
import numpy as np
from pydub import AudioSegment
# 初始化， 必须要有奥
engine = pyttsx3.init()

In [25]:
import pyttsx3

tts = pyttsx3.init()
voices = tts.getProperty('voices')
for voice in voices:
    print('id = {} \nname = {} \n'.format(voice.id, voice.name))

id = HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_ZH-CN_HUIHUI_11.0 
name = Microsoft Huihui Desktop - Chinese (Simplified) 

id = HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0 
name = Microsoft Zira Desktop - English (United States) 



In [26]:
rate = engine.getProperty('rate')   # getting details of current speaking rate
print (rate)      
volume = engine.getProperty('volume')
print(volume)
voices = engine.getProperty('voices') 
print(voices)
engine.setProperty('voices',voices[1].id)

250
1.0
[<pyttsx3.voice.Voice object at 0x0000017B8DACE110>, <pyttsx3.voice.Voice object at 0x0000017B8DAE89A0>]


In [27]:
text=["科技并带着趣味",
"#","不觉得这很酷吗",
"#","作为一名理工男",
"#","我觉得这太酷了",
"#","很符合我对未来生活的想象"]
text_dir={}
for i in text:
    text_dir[i]=200

text_dir["很符合我对未来生活的想象"]=250
text_dir["科技并带着趣味"]=210
text_dir["这太酷了"]=50
text_dir["我觉得"]=300
text_dir["作为一名理工男"]=250
text_dir["不觉得"]=220

In [28]:
# text=["科技并带着趣味",
# "不觉得这很酷吗",
# "作为一名理工男",
# "我觉得这太酷了",
# "很符合我对未来生活的想象"]

FORMAT = ['wav','pcm','ogg','mp3','m4a','aac']
ext='.aac'
dir="vio"
if not os.path.exists(dir):
    os.mkdir(dir)

In [29]:
#空白
blank_time=[-1325,-4360+3315,-6120+5230,-12120+8820+10000-7520]
i=0
for t in blank_time:
    slience=AudioSegment.silent(10*1000+t)
    slience.export(os.path.join(dir,"blank"+str(i)+".mp3"))
    i=i+1

In [30]:
#获得音频
sen_list=["不觉得这很酷吗","科技并带着趣味","我觉得这太酷了"]
for sen in sen_list:
    if os.path.exists(os.path.join(dir,sen+'.mp3')):
        os.remove(os.path.join(dir,sen+'.mp3'))
    os.system('ffmpeg -n -i '+sen+'.mp3 -filter:a "atempo=1.4" '+os.path.join(dir,sen+'.mp3'))

engine.setProperty('volume',1)

for sen in text:
    if sen in sen_list:
        print(sen)
        continue
    engine.setProperty('rate',text_dir[sen])
    engine.save_to_file(sen, os.path.join(dir,sen+".mp3"))
    engine.runAndWait()

科技并带着趣味
不觉得这很酷吗
我觉得这太酷了


In [31]:
#压缩
res=[]
temp=text[:]
i=0
for sen in temp:
    if sen=="#":
        ori=os.path.join(dir,"blank"+str(i)+".mp3")
        obj=os.path.join(dir,"blank"+str(i)+"_"+ext)
        res.append(obj)
        print("ffmpeg -y -i "+ori+" -b:a 4k -ar 4000 "+obj)
        os.system("ffmpeg -y -i "+ori+" -b:a 8k -ar 8000 "+obj)

        i=i+1
        continue
    ori=os.path.join(dir,sen+".mp3")
    obj=os.path.join(dir,sen+"1"+ext)
    res.append(obj)
    print("ffmpeg -y -i "+ori+" -b:a 8k -ar 8000 "+obj)
    os.system("ffmpeg -y -i "+ori+" -b:a 8k -ar 8000 "+obj)
        

ffmpeg -y -i test\科技并带着趣味.mp3 -b:a 8k -ar 8000 test\科技并带着趣味1.ogg
ffmpeg -y -i test\#.mp3 -b:a 8k -ar 8000 test\#1.ogg
ffmpeg -y -i test\不觉得这很酷吗.mp3 -b:a 8k -ar 8000 test\不觉得这很酷吗1.ogg
ffmpeg -y -i test\#.mp3 -b:a 8k -ar 8000 test\#1.ogg
ffmpeg -y -i test\作为一名理工男.mp3 -b:a 8k -ar 8000 test\作为一名理工男1.ogg
ffmpeg -y -i test\#.mp3 -b:a 8k -ar 8000 test\#1.ogg
ffmpeg -y -i test\我觉得这太酷了.mp3 -b:a 8k -ar 8000 test\我觉得这太酷了1.ogg
ffmpeg -y -i test\#.mp3 -b:a 8k -ar 8000 test\#1.ogg
ffmpeg -y -i test\很符合我对未来生活的想象.mp3 -b:a 8k -ar 8000 test\很符合我对未来生活的想象1.ogg


In [32]:
l1 = res[:]

cmd = ('ffmpeg -y -i "concat:%s" -acodec copy %s' % ('|'.join(l1), 'res1'+ext))
print(cmd)
os.popen(cmd)

ffmpeg -y -i "concat:test\科技并带着趣味1.ogg|test\#1.ogg|test\不觉得这很酷吗1.ogg|test\#1.ogg|test\作为一名理工男1.ogg|test\#1.ogg|test\我觉得这太酷了1.ogg|test\#1.ogg|test\很符合我对未来生活的想象1.ogg" -acodec copy res1.ogg
